## Task: Question Answering for Game of Thrones

<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/en/d/d8/Game_of_Thrones_title_card.jpg">

Question Answering can be used in a variety of use cases. A very common one:  Using it to navigate through complex knowledge bases or long documents ("search setting").

A "knowledge base" could for example be your website, an internal wiki or a collection of financial reports. 
In this tutorial we will work on a slightly different domain: "Game of Thrones". 

Let's see how we can use a bunch of wikipedia articles to answer a variety of questions about the 
marvellous seven kingdoms...  


In [1]:
# Let's start by adjust the working directory so that it is the root of the repository
# This should be run just once.
import os
os.chdir('../')
print("Current working directory is {}".format(os.getcwd()))

Current working directory is /home/mp/deepset/dev/haystack


In [2]:
from haystack.reader.farm import FARMReader
from haystack.retriever.tfidf import TfidfRetriever
from haystack import Finder
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.indexing.cleaning import clean_wiki_text
from haystack.utils import print_answers

I1125 16:55:41.544814 139975239116608 file_utils.py:39] PyTorch version 1.3.0 available.
I1125 16:55:41.619155 139975239116608 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


## Indexing & cleaning documents

In [3]:
# Init a database (default: sqllite)
from haystack.database import db
db.create_all()

# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Now, let's write the docs to our DB. 
# You can supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
write_documents_to_db(document_dir=doc_dir, clean_func=clean_wiki_text)

I1125 16:55:41.863932 139975239116608 io.py:57] Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1167348/1167348 [00:00<00:00, 9196388.44B/s]
I1125 16:55:47.962270 139975239116608 io.py:30] Wrote 517 docs to DB


## Initalize Reader, Retriever & Finder

In [4]:
# A retriever identifies the k most promising chunks of text that might contain the answer for our question
# Retrievers use some simple but fast algorithm, here: TF-IDF
retriever = TfidfRetriever()

I1125 16:55:48.018222 139975239116608 tfidf.py:69] Found 2811 candidate paragraphs from 517 docs in DB


In [5]:
# A reader scans the text chunks in detail and extracts the k best answers
# Reader use more powerful but slower deep learning models, here: a BERT QA model trained via FARM on Squad 2.0
reader = FARMReader(model_dir="../FARM/saved_models/bert-english-qa-large", use_gpu=False)

W1125 16:55:53.613250 139975239116608 processor.py:170] Loading tokenizer from deprecated FARM config. If you used `custom_vocab` or `never_split_chars`, this won't work anymore.


In [6]:
# The Finder sticks together retriever and retriever in a pipeline to answer our actual questions 
finder = Finder(reader, retriever)

## Voilá! Ask a question!

In [7]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)

I1125 16:55:54.057870 139975239116608 __init__.py:92] Identified 10 candidates via retriever:
  paragraph_id  document_id                                                                             text
         1257          227  \n===Arya Stark===\n'''Arya Stark''' portrayed by Maisie Williams. Arya Star...
         1023          169  \n====Season 8====\nArya reunites with Jon, Gendry, and the Hound, who have ...
         1016          169  \n====Season 1====\nArya accompanies her father Ned and her sister Sansa to ...
          718          144  \n===''A Game of Thrones''===\nSansa Stark begins the novel by being betroth...
          161           33  \n===In Braavos===\nLady Crane returns to her chambers to find a wounded Ary...
         1846          304  \n== Characters ==\nThe tale is told through the eyes of 9 recurring POV cha...
         1009          169  \n==== ''A Game of Thrones'' ====\nArya adopts a direwolf cub, which she nam...
         1022          169  \n====Season 

In [8]:
#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_reader=5)

In [9]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eddard',
        'context': 'ry warrior queen. She travels with her father, Eddard, to '
                   "King's Landing when he is made Hand of the"},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before the"},
    {   'answer': 'Lord Eddard',
        'context': ' is the younger daughter and third child of Lord Eddard '
                   'and Catelyn Stark of Winterfell. Ever the to'},
    {   'answer': 'Lord Eddard Stark',
        'context': ' Tourney of the Hand to honour her father Lord Eddard '
                   'Stark, Sansa Stark is enchanted by the knights'},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': 'e third child and younger daughter of Eddard and Catelyn '
                   'Stark. She serves as a POV character for 33'}]
